In [4]:
import pandas as pd
from surprise import Reader, SVD, Dataset, accuracy
from surprise.model_selection import train_test_split, cross_validate
from rs_datasets import MovieLens
import pickle

log_df_train = pd.read_csv('train/rating.csv').drop('timestamp', axis = 1)

In [5]:
reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(log_df_train, reader)
data = data.build_full_trainset()
# trainset, testset = train_test_split(data, test_size=.25)

svd_model = SVD(n_factors=64)
svd_model.fit(data)

# predictions = svd_model.test(testset)
# print(accuracy.rmse(predictions))
# print(accuracy.mae(predictions))

In [22]:
user_cols = [f'uembed_{i}' for i in range(64)]
user_features = pd.DataFrame(svd_model.pu, columns=user_cols)
user_features['user_idx'] = sorted(log_df_train['user_idx'].unique())
user_cols.insert(0, 'user_idx')
user_features = user_features[user_cols]

item_cols = [f'iembed_{i}' for i in range(64)]
item_features = pd.DataFrame(svd_model.qi, columns=item_cols)
item_features['item_idx'] = sorted(log_df_train['item_idx'].unique())
item_cols.insert(0, 'item_idx')
item_features = item_features[item_cols]

In [9]:
user_features.to_csv('user_svd.csv', index=False)
item_features.to_csv('item_svd.csv', index=False)